In [1]:
import pandas as pd
from langdetect import detect
import re

def clean_twitter_data(file_path):
    # อ่านไฟล์ Excel
    df = pd.read_excel(file_path)

    # ฟังก์ชันสำหรับทำความสะอาดข้อความ
    def clean_text(text):
        if isinstance(text, str):
            # ลบ URL
            text = re.sub(r'http\S+|www.\S+', '', text)
            # ลบช่องว่างที่ไม่จำเป็น
            text = ' '.join(text.split())
            # ลบเครื่องหมายพิเศษตามที่ต้องการ
            text = text.strip()
            return text
        return ''

    # ฟังก์ชันสำหรับดึง hashtags
    def extract_hashtags(text):
        if isinstance(text, str):
            hashtags = re.findall(r'#(\w+)', text)
            return set(hashtags)
        return set()

    # ฟังก์ชันตรวจสอบภาษาอังกฤษ
    def is_english(text):
        try:
            if isinstance(text, str):
                return detect(text) == 'en'
            return False
        except:
            return False

    # ทำความสะอาดข้อมูล
    df['clean_text'] = df['tweetText'].apply(clean_text)
    df['hashtag_set'] = df['tweetText'].apply(extract_hashtags)

    # กรองเฉพาะภาษาอังกฤษ
    df = df[df['clean_text'].apply(is_english)]

    # รวมทวีตที่ซ้ำกัน
    grouped = df.groupby('clean_text', as_index=False).agg({
        'tweetText': 'first',
        'hashtag_set': lambda x: set.union(*x)
    })

    # แปลง hashtag_set เป็น string
    grouped['hashtags'] = grouped['hashtag_set'].apply(lambda x: ','.join(sorted(x)) if x else '')

    # เลือกคอลัมน์ที่ต้องการและบันทึก
    result_df = grouped[['tweetText', 'hashtags']]

    # เรียงลำดับข้อมูลตาม tweetText เพื่อให้ง่ายต่อการตรวจสอบ
    result_df = result_df.sort_values('tweetText')

    output_path = 'cholera-clean.xlsx'  # เปลี่ยนชื่อไฟล์เพื่อไม่ให้ทับไฟล์เดิม
    result_df.to_excel(output_path, index=False)
    print(f"บันทึกข้อมูลที่ทำความสะอาดแล้วไปที่ {output_path}")
    print(f"จำนวนทวีตก่อนรวม: {len(df)}")
    print(f"จำนวนทวีตหลังรวม: {len(result_df)}")
    return result_df

if __name__ == "__main__":
    file_path = "cholera.xlsx"
    cleaned_data = clean_twitter_data(file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'cholera.xlsx'